## GPT-2を芥川龍之介の作品を用いてファインチューニングする

In [ ]:
# Google Driveをマウントと作業用ディレクトリ作成
from google.colab import drive 
drive.mount('/content/drive')
!mkdir -p '/content/drive/My Drive/work/'
# ファインチューニングした結果を保存するためのディレクトリ作成
!mkdir -p '/content/drive/MyDrive/work/output/'
%cd '/content/drive/My Drive/work/'

In [ ]:
# 学習データをダウンロード
!wget https://raw.githubusercontent.com/koit2525/Fake_Akutagawa_sentence/main/train.txt

In [ ]:
# transformersをインストール
!pip install git+https://github.com/huggingface/transformers
# rinna/japanese-gpt2-mediumのトークナイザーはsentencepieceなのでsentencepieceもインストール
!pip install folium==0.2.1
!pip install sentencepiece
!pip install datasets

In [ ]:
# ファインチューニングにrun_clm.py利用するので、transformaersをダウンロード
!git clone https://github.com/huggingface/transformers

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("rinna/japanese-gpt2-medium")
tokenizer.do_lower_case = True
model = AutoModelForCausalLM.from_pretrained("rinna/japanese-gpt2-medium")

In [ ]:
# 学習データを用いてファインチューニングを実施
!python ./transformers/examples/pytorch/language-modeling/run_clm.py \
    --model_name_or_path=rinna/japanese-gpt2-medium \
    --train_file=/content/drive/MyDrive/work/train.txt \
    --validation_file=/content/drive/MyDrive/work/train.txt \
    --do_train \
    --do_eval \
    --num_train_epochs=10 \
    --save_steps=10000 \
    --save_total_limit=3 \
    --per_device_train_batch_size=1 \
    --per_device_eval_batch_size=1 \
    --output_dir=/content/drive/MyDrive/work/output/ \
    --use_fast_tokenizer=False

## ファインチューニングしたモデルを利用して文章を生成する

In [ ]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ファインチューニングしたモデルを読み込む
model = AutoModelForCausalLM.from_pretrained("/content/drive/MyDrive/work/output")
model.to(device)
model.eval()

In [ ]:
# 生成文の文頭となる文章
input_texts = ["ある日の暮方の事である。", "ある日の事でございます。", "僕の将来に対する唯ぼんやりとした不安",]

for text in input_texts:
  input_ids = tokenizer.encode(text, return_tensors='pt').to(device)
  out = model.generate(input_ids, do_sample=True, min_length=100, max_length=400, pad_token_id=tokenizer.eos_token_id)
  sent = tokenizer.batch_decode(out)[0]
  sent = sent.replace('</s>', '')
  sent = sent.replace('<unk>', '')
  sent = sent.replace('。', '。\n')
  print(f"-------「{text}」から生成した芥川龍之介風文章-------")
  print(f"{sent.strip()}\n")